# Training Models


Imports


In [1]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.metrics import (
    classification_report,
    precision_score,
    recall_score,
    accuracy_score,
    f1_score,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, Dropout
from imblearn.over_sampling import SMOTE

In [2]:
df = pd.read_csv("../Data/cleaned_dataset.csv")

In [3]:
X = df.iloc[:, 2:-1]
y = df.iloc[:, -1]

In [4]:
sm = SMOTE()

In [5]:
X_res, y_res = sm.fit_resample(X, y)

In [6]:
df = pd.concat([X_res, y_res], axis=1)

In [7]:
train_df, test_df = train_test_split(df, test_size=0.2)

Get the training features and label


In [8]:
X_train = train_df.iloc[:, :-1].to_numpy()
y_train = train_df.iloc[:, -1].to_numpy()

Get the testing features and label


In [9]:
X_test = test_df.iloc[:, :-1].to_numpy()
y_test = test_df.iloc[:, -1].to_numpy()

Initialize a 5-fold cross-validation object


In [10]:
KF = KFold(n_splits=5)

## KNN


In [11]:
KNN = KNeighborsClassifier()

In [12]:
param_grid = {"n_neighbors": list(range(1, int(np.sqrt(len(X_train))), 2))}

In [13]:
grid_search = GridSearchCV(estimator=KNN, param_grid=param_grid, cv=KF, verbose=2)

In [14]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 58 candidates, totalling 290 fits
[CV] END ......................................n_neighbors=1; total time=   0.2s
[CV] END ......................................n_neighbors=1; total time=   0.2s
[CV] END ......................................n_neighbors=1; total time=   0.3s
[CV] END ......................................n_neighbors=1; total time=   0.2s
[CV] END ......................................n_neighbors=1; total time=   0.2s
[CV] END ......................................n_neighbors=3; total time=   0.2s
[CV] END ......................................n_neighbors=3; total time=   0.2s
[CV] END ......................................n_neighbors=3; total time=   0.2s
[CV] END ......................................n_neighbors=3; total time=   0.2s
[CV] END ......................................n_neighbors=3; total time=   0.2s
[CV] END ......................................n_neighbors=5; total time=   0.2s
[CV] END ......................................

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21,
                                         23, 25, 27, 29, 31, 33, 35, 37, 39, 41,
                                         43, 45, 47, 49, 51, 53, 55, 57, 59, ...]},
             verbose=2)

In [15]:
best_params = grid_search.best_params_
print(best_params)

{'n_neighbors': 1}


In [16]:
KNN_best = KNeighborsClassifier(**best_params)

In [17]:
KNN_best.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=1)

In [18]:
predictions = KNN_best.predict(X_test)

In [19]:
print(classification_report(y_true=y_test, y_pred=predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.90      0.80      0.85      1704
           1       0.82      0.91      0.86      1670

    accuracy                           0.86      3374
   macro avg       0.86      0.86      0.86      3374
weighted avg       0.86      0.86      0.86      3374



## Naive Bayes


In [20]:
NB = GaussianNB()

In [21]:
precisions = []
recalls = []
accuracies = []
f1_scores = []

In [22]:
fold = 1
for train, test in KF.split(X_train, y_train):
    print(f"##### FOLD: {fold} #####")

    # Fit the model
    NB.fit(X_train[train], y_train[train])

    # Predict on the test set
    predictions = NB.predict(X_train[test])

    # Evaluate the model
    precision = precision_score(
        y_true=y_train[test],
        y_pred=predictions,
        zero_division=0,
        average="weighted",
    )
    recall = recall_score(
        y_true=y_train[test],
        y_pred=predictions,
        zero_division=0,
        average="weighted",
    )
    accuracy = accuracy_score(y_true=y_train[test], y_pred=predictions)
    f1 = f1_score(
        y_true=y_train[test],
        y_pred=predictions,
        zero_division=0,
        average="weighted",
    )

    # Store the result
    precisions.append(precision)
    recalls.append(recall)
    accuracies.append(accuracy)
    f1_scores.append(f1)

    # Print the scores for each fold
    print(f"Precision = {precision}")
    print(f"Recall = {recall}")
    print(f"Accuracy = {accuracy}")
    print(f"F1 score = {f1}\n")

    fold += 1

##### FOLD: 1 #####
Precision = 0.759998164409735
Recall = 0.6795109299740645
Accuracy = 0.6795109299740645
F1 score = 0.6525732561653239

##### FOLD: 2 #####
Precision = 0.7581739186885681
Recall = 0.6676546869210819
Accuracy = 0.6676546869210819
F1 score = 0.6340714745053027

##### FOLD: 3 #####
Precision = 0.7645022102893884
Recall = 0.6738324684951816
Accuracy = 0.6738324684951816
F1 score = 0.6423329762504331

##### FOLD: 4 #####
Precision = 0.7502061100247954
Recall = 0.6734618235730171
Accuracy = 0.6734618235730171
F1 score = 0.6437107831921597

##### FOLD: 5 #####
Precision = 0.75455228101848
Recall = 0.6730911786508524
Accuracy = 0.6730911786508524
F1 score = 0.6477599805640614



In [23]:
print("Mean Scores:")
print(f"Mean Precision = {np.mean(precisions)}")
print(f"Mean Recall = {np.mean(recalls)}")
print(f"Mean Accuracy = {np.mean(accuracies)}")
print(f"Mean F1 score = {np.mean(f1_scores)}")

Mean Scores:
Mean Precision = 0.7574865368861934
Mean Recall = 0.6735102175228395
Mean Accuracy = 0.6735102175228395
Mean F1 score = 0.6440896941354561


In [24]:
predictions = NB.predict(X_test)

In [25]:
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.90      0.42      0.57      1704
           1       0.62      0.95      0.75      1670

    accuracy                           0.68      3374
   macro avg       0.76      0.69      0.66      3374
weighted avg       0.76      0.68      0.66      3374



## CART


In [26]:
Cart = DecisionTreeClassifier()

In [27]:
param_grid = {
    "max_depth": [None, 10, 20, 30],
    "class_weight": [
        None,
        {0: 1, 1: 2},
        {0: 1, 1: 3},
        {0: 1.5, 1: 2.5},
        {0: 1.5, 1: 3},
    ],
    "ccp_alpha": [0, 1, 0.01, 0.001],
    "min_samples_split": [2, 3, 4],
}

In [28]:
grid_search = GridSearchCV(estimator=Cart, param_grid=param_grid, cv=KF, verbose=2)

In [29]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 240 candidates, totalling 1200 fits
[CV] END ccp_alpha=0, class_weight=None, max_depth=None, min_samples_split=2; total time=   0.3s
[CV] END ccp_alpha=0, class_weight=None, max_depth=None, min_samples_split=2; total time=   0.3s
[CV] END ccp_alpha=0, class_weight=None, max_depth=None, min_samples_split=2; total time=   0.3s
[CV] END ccp_alpha=0, class_weight=None, max_depth=None, min_samples_split=2; total time=   0.3s
[CV] END ccp_alpha=0, class_weight=None, max_depth=None, min_samples_split=2; total time=   0.3s
[CV] END ccp_alpha=0, class_weight=None, max_depth=None, min_samples_split=3; total time=   0.3s
[CV] END ccp_alpha=0, class_weight=None, max_depth=None, min_samples_split=3; total time=   0.3s
[CV] END ccp_alpha=0, class_weight=None, max_depth=None, min_samples_split=3; total time=   0.3s
[CV] END ccp_alpha=0, class_weight=None, max_depth=None, min_samples_split=3; total time=   0.3s
[CV] END ccp_alpha=0, class_weight=None, max_depth=None, min_sa

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=DecisionTreeClassifier(),
             param_grid={'ccp_alpha': [0, 1, 0.01, 0.001],
                         'class_weight': [None, {0: 1, 1: 2}, {0: 1, 1: 3},
                                          {0: 1.5, 1: 2.5}, {0: 1.5, 1: 3}],
                         'max_depth': [None, 10, 20, 30],
                         'min_samples_split': [2, 3, 4]},
             verbose=2)

In [30]:
best_params = grid_search.best_params_
print(best_params)

{'ccp_alpha': 0, 'class_weight': None, 'max_depth': 10, 'min_samples_split': 3}


In [31]:
Cart_best = DecisionTreeClassifier(**best_params)

In [32]:
Cart_best.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0, max_depth=10, min_samples_split=3)

In [33]:
predictions = Cart_best.predict(X_test)

In [34]:
print(classification_report(y_true=y_test, y_pred=predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.86      0.86      0.86      1704
           1       0.86      0.85      0.86      1670

    accuracy                           0.86      3374
   macro avg       0.86      0.86      0.86      3374
weighted avg       0.86      0.86      0.86      3374



## Random Forest


In [35]:
RF = RandomForestClassifier(n_jobs=5)

In [36]:
param_grid = {
    "n_estimators": [50, 100, 200, 300, 400],
    "max_depth": [None, 10, 20, 30],
    "class_weight": [
        None,
        "balanced",
        "balanced_subsample",
        {0: 1, 1: 2},
        {0: 1, 1: 3},
        {0: 1.5, 1: 2.5},
        {0: 1.5, 1: 3},
    ],
    "ccp_alpha": [0, 1, 0.01, 0.001],
}

In [37]:
grid_search = GridSearchCV(estimator=RF, param_grid=param_grid, cv=KF, verbose=2)

In [38]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 560 candidates, totalling 2800 fits
[CV] END ccp_alpha=0, class_weight=None, max_depth=None, n_estimators=50; total time=   0.2s
[CV] END ccp_alpha=0, class_weight=None, max_depth=None, n_estimators=50; total time=   0.2s
[CV] END ccp_alpha=0, class_weight=None, max_depth=None, n_estimators=50; total time=   0.2s
[CV] END ccp_alpha=0, class_weight=None, max_depth=None, n_estimators=50; total time=   0.2s
[CV] END ccp_alpha=0, class_weight=None, max_depth=None, n_estimators=50; total time=   0.2s
[CV] END ccp_alpha=0, class_weight=None, max_depth=None, n_estimators=100; total time=   0.4s
[CV] END ccp_alpha=0, class_weight=None, max_depth=None, n_estimators=100; total time=   0.4s
[CV] END ccp_alpha=0, class_weight=None, max_depth=None, n_estimators=100; total time=   0.4s
[CV] END ccp_alpha=0, class_weight=None, max_depth=None, n_estimators=100; total time=   0.3s
[CV] END ccp_alpha=0, class_weight=None, max_depth=None, n_estimators=100; total time=   0.3s
[

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=RandomForestClassifier(n_jobs=5),
             param_grid={'ccp_alpha': [0, 1, 0.01, 0.001],
                         'class_weight': [None, 'balanced',
                                          'balanced_subsample', {0: 1, 1: 2},
                                          {0: 1, 1: 3}, {0: 1.5, 1: 2.5},
                                          {0: 1.5, 1: 3}],
                         'max_depth': [None, 10, 20, 30],
                         'n_estimators': [50, 100, 200, 300, 400]},
             verbose=2)

In [39]:
best_params = grid_search.best_params_
print(best_params)

{'ccp_alpha': 0, 'class_weight': {0: 1, 1: 3}, 'max_depth': None, 'n_estimators': 400}


In [40]:
RF_best = RandomForestClassifier(**best_params)

In [41]:
RF_best.fit(X_train, y_train)

RandomForestClassifier(ccp_alpha=0, class_weight={0: 1, 1: 3}, n_estimators=400)

In [42]:
predictions = RF_best.predict(X_test)

In [43]:
print(classification_report(y_true=y_test, y_pred=predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.87      0.91      0.89      1704
           1       0.90      0.86      0.88      1670

    accuracy                           0.89      3374
   macro avg       0.89      0.88      0.88      3374
weighted avg       0.89      0.89      0.88      3374



## Logistic Regression


In [44]:
LR = LogisticRegression(n_jobs=5)

In [45]:
param_grid = {
    "tol": [0.001, 0.0001, 0.00001],
    "solver": ["lbfgs", "liblinear"],
    "max_iter": [2000, 6000, 10000],
    "class_weight": [
        None,
        {0: 1, 1: 2},
        {0: 1, 1: 3},
        {0: 1.5, 1: 2.5},
        {0: 1.5, 1: 3},
    ],
}

In [46]:
grid_search = GridSearchCV(estimator=LR, param_grid=param_grid, cv=KF, verbose=2)

In [47]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 90 candidates, totalling 450 fits


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END class_weight=None, max_iter=2000, solver=lbfgs, tol=0.001; total time=   5.1s


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END class_weight=None, max_iter=2000, solver=lbfgs, tol=0.001; total time=   4.9s


...

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1216: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 5.
  warnings.warn(


[CV] END class_weight={0: 1.5, 1: 3}, max_iter=10000, solver=liblinear, tol=1e-05; total time=  10.4s


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1216: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 5.
  warnings.warn(


[CV] END class_weight={0: 1.5, 1: 3}, max_iter=10000, solver=liblinear, tol=1e-05; total time=  17.3s


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=LogisticRegression(n_jobs=5),
             param_grid={'class_weight': [None, {0: 1, 1: 2}, {0: 1, 1: 3},
                                          {0: 1.5, 1: 2.5}, {0: 1.5, 1: 3}],
                         'max_iter': [2000, 6000, 10000],
                         'solver': ['lbfgs', 'liblinear'],
                         'tol': [0.001, 0.0001, 1e-05]},
             verbose=2)

In [48]:
best_params = grid_search.best_params_
print(best_params)

{'class_weight': None, 'max_iter': 10000, 'solver': 'lbfgs', 'tol': 1e-05}


In [49]:
LR_best = LogisticRegression(**best_params)

In [50]:
LR_best.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=10000, tol=1e-05)

In [51]:
predictions = LR_best.predict(X_test)

In [52]:
print(classification_report(y_true=y_test, y_pred=predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.88      0.88      0.88      1704
           1       0.88      0.87      0.88      1670

    accuracy                           0.88      3374
   macro avg       0.88      0.88      0.88      3374
weighted avg       0.88      0.88      0.88      3374



## SVM


In [53]:
SVM = SVC()

In [54]:
param_grid = {
    "tol": [0.001, 0.0001, 0.00001],
    "kernel": ["poly", "rbf", "sigmoid"],
    "class_weight": [
        None,
        {0: 1, 1: 2},
        {0: 1, 1: 2.5},
        {0: 1.5, 1: 2.5},
        {0: 1, 1: 3},
    ],
}

In [55]:
grid_search = GridSearchCV(estimator=SVM, param_grid=param_grid, cv=KF, verbose=2)

In [56]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END ..........class_weight=None, kernel=poly, tol=0.001; total time=  28.3s
[CV] END ..........class_weight=None, kernel=poly, tol=0.001; total time=  28.2s
[CV] END ..........class_weight=None, kernel=poly, tol=0.001; total time=  27.8s
[CV] END ..........class_weight=None, kernel=poly, tol=0.001; total time=  27.3s
[CV] END ..........class_weight=None, kernel=poly, tol=0.001; total time=  27.6s
[CV] END .........class_weight=None, kernel=poly, tol=0.0001; total time=  28.0s
[CV] END .........class_weight=None, kernel=poly, tol=0.0001; total time=  28.1s
[CV] END .........class_weight=None, kernel=poly, tol=0.0001; total time=  27.7s
[CV] END .........class_weight=None, kernel=poly, tol=0.0001; total time=  27.4s
[CV] END .........class_weight=None, kernel=poly, tol=0.0001; total time=  27.3s
[CV] END ..........class_weight=None, kernel=poly, tol=1e-05; total time=  27.8s
[CV] END ..........class_weight=None, kernel=po

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=SVC(),
             param_grid={'class_weight': [None, {0: 1, 1: 2}, {0: 1, 1: 2.5},
                                          {0: 1.5, 1: 2.5}, {0: 1, 1: 3}],
                         'kernel': ['poly', 'rbf', 'sigmoid'],
                         'tol': [0.001, 0.0001, 1e-05]},
             verbose=2)

In [57]:
best_params = grid_search.best_params_
print(best_params)

{'class_weight': None, 'kernel': 'rbf', 'tol': 0.001}


In [58]:
SVM_best = SVC(**best_params)

In [59]:
SVM_best.fit(X_train, y_train)

SVC()

In [60]:
predictions = SVM_best.predict(X_test)

In [61]:
print(classification_report(y_true=y_test, y_pred=predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.86      0.56      0.68      1704
           1       0.67      0.91      0.77      1670

    accuracy                           0.73      3374
   macro avg       0.77      0.73      0.73      3374
weighted avg       0.77      0.73      0.73      3374



## Neural Network


In [62]:
early_stopping = EarlyStopping(monitor="val_loss", patience=10)

In [63]:
nn = Sequential()
nn.add(Dense(128, activation="relu"))
nn.add(Dropout(0.2))
nn.add(Dense(1, activation="sigmoid"))

nn.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [64]:
precisions = []
recalls = []
accuracies = []
f1_scores = []

In [65]:
fold = 1
for train, test in KF.split(X_train, y_train):
    print(f"##### FOLD: {fold} #####")

    # Fit the model
    nn.fit(
        X_train[train],
        y_train[train],
        epochs=200,
        batch_size=32,
        validation_data=(X_train[test], y_train[test]),
        callbacks=[early_stopping],
        verbose=0,
    )

    # Predict on the test set
    y_pred = nn.predict(X_train[test])

    predictions = []
    for pred in y_pred:
        if pred > 0.5:
            predictions.append(1)
        else:
            predictions.append(0)

    # Evaluate the model
    precision = precision_score(
        y_true=y_train[test],
        y_pred=predictions,
        zero_division=0,
        average="weighted",
    )
    recall = recall_score(
        y_true=y_train[test],
        y_pred=predictions,
        zero_division=0,
        average="weighted",
    )
    accuracy = accuracy_score(y_true=y_train[test], y_pred=predictions)
    f1 = f1_score(
        y_true=y_train[test],
        y_pred=predictions,
        zero_division=0,
        average="weighted",
    )

    # Store the result
    precisions.append(precision)
    recalls.append(recall)
    accuracies.append(accuracy)
    f1_scores.append(f1)

    # Print the scores for each fold
    print(f"Precision = {precision}")
    print(f"Recall = {recall}")
    print(f"Accuracy = {accuracy}")
    print(f"F1 score = {f1}\n")

    fold += 1

##### FOLD: 1 #####
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 438us/step
Precision = 0.8903568602935666
Recall = 0.8899592441645053
Accuracy = 0.8899592441645053
F1 score = 0.8899319562264341

##### FOLD: 2 #####
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 389us/step
Precision = 0.9262658522607405
Recall = 0.9255279733234532
Accuracy = 0.9255279733234532
F1 score = 0.9255122278810454

##### FOLD: 3 #####
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 336us/step
Precision = 0.939965809764297
Recall = 0.9399555226093402
Accuracy = 0.9399555226093402
F1 score = 0.9399560175423215

##### FOLD: 4 #####
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 517us/step
Precision = 0.9584813812414573
Recall = 0.958117123795404
Accuracy = 0.958117123795404
F1 score = 0.95811903982262

##### FOLD: 5 #####
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 374us/step
Precision = 0.9671152518488805
Recall = 0.9670126019273536
Accuracy = 0.9670126019273536
F1 score = 0.9670052681440602



In [66]:
print("Mean Scores:")
print(f"Mean Precision = {np.mean(precisions)}")
print(f"Mean Recall = {np.mean(recalls)}")
print(f"Mean Accuracy = {np.mean(accuracies)}")
print(f"Mean F1 score = {np.mean(f1_scores)}")

Mean Scores:
Mean Precision = 0.9364370310817884
Mean Recall = 0.9361144931640112
Mean Accuracy = 0.9361144931640112
Mean F1 score = 0.9361049019232961


In [67]:
y_pred = nn.predict(X_test)

predictions = []
for pred in y_pred:
    if pred > 0.5:
        predictions.append(1)
    else:
        predictions.append(0)

  1/106 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step

106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 386us/step


In [68]:
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.89      0.91      0.90      1704
           1       0.91      0.88      0.89      1670

    accuracy                           0.90      3374
   macro avg       0.90      0.90      0.90      3374
weighted avg       0.90      0.90      0.90      3374

